In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import platform
from matplotlib import font_manager, rc

# 한글
if platform.system() == "Windows":
    font_name = font_manager.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
    rc('font', family = font_name)
elif platform.system() == "Darwin":
    rc('font', family = 'AppleGothic')

plt.rcParams['axes.unicode_minus'] = False

In [3]:
# auto_mpg 데이터를 가져와서 horsepower 의 이상치를 제거하고 숫자 타입으로 변환
auto_mpg = pd.read_csv('./data4/auto-mpg.csv', header = None)
auto_mpg.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin','name']

auto_mpg['horsepower'].replace('?', np.nan, inplace = True)
auto_mpg.dropna(subset = ['horsepower'], axis = 0, inplace = True)
auto_mpg['horsepower'] = auto_mpg['horsepower'].astype('float')
print(auto_mpg.head())

    mpg  cylinders  displacement  horsepower  weight  acceleration  \
0  18.0          8         307.0       130.0  3504.0          12.0   
1  15.0          8         350.0       165.0  3693.0          11.5   
2  18.0          8         318.0       150.0  3436.0          11.0   
3  16.0          8         304.0       150.0  3433.0          12.0   
4  17.0          8         302.0       140.0  3449.0          10.5   

   model year  origin                       name  
0          70       1  chevrolet chevelle malibu  
1          70       1          buick skylark 320  
2          70       1         plymouth satellite  
3          70       1              amc rebel sst  
4          70       1                ford torino  


In [7]:
# 원 핫 인코딩
# horsepower 특성을 범주형으로 추가 - 3개의 영역으로 구분
count, bin_dividers = np.histogram(auto_mpg['horsepower'], bins = 3)

# 각 그룹에 할당할 값의 리스트
bin_names = ['저출력', '보통출력', '고출력']
auto_mpg['hp_bin'] = pd.cut(x = auto_mpg['horsepower'], bins = bin_dividers, labels = bin_names, include_lowest =True)
# print(auto_mpg[['horsepower','hp_bin']].head(20))

# 원 핫 인코딩 수행 - 값이 3종류 이므로 3개의 특성이 만들어지고 값은 하나만 1이 됨
horsepower_dummies = pd.get_dummies(auto_mpg['hp_bin'])
print(horsepower_dummies)

     저출력  보통출력  고출력
0      0     1    0
1      0     1    0
2      0     1    0
3      0     1    0
4      0     1    0
..   ...   ...  ...
393    1     0    0
394    1     0    0
395    1     0    0
396    1     0    0
397    1     0    0

[392 rows x 3 columns]


In [12]:
# 하나의 특성을 원 핫 인코딩해주는 클래스
from sklearn.preprocessing import LabelBinarizer

one_hot = LabelBinarizer()
print(one_hot.fit_transform(auto_mpg['hp_bin']))

# 이름을 확인
print(one_hot.classes_)

[[0 1 0]
 [0 1 0]
 [0 1 0]
 ...
 [0 0 1]
 [0 0 1]
 [0 0 1]]
['고출력' '보통출력' '저출력']


In [13]:
# 2개 이상의 특성을 가지고 원 핫 인코딩
# 2개 이상의 1이 등장할 수 있음
from sklearn.preprocessing import MultiLabelBinarizer

multi_features = [('java','C++'), ('C++','Python'), ('java','C#'),
                  ('java','Kotlin'), ('Python','Go'), ('Python','R')]

one_hot_multiclass = MultiLabelBinarizer()

print(one_hot_multiclass.fit_transform(multi_features))
print(one_hot_multiclass.classes_)


[[0 1 0 0 0 0 1]
 [0 1 0 0 1 0 0]
 [1 0 0 0 0 0 1]
 [0 0 0 1 0 0 1]
 [0 0 1 0 1 0 0]
 [0 0 0 0 1 1 0]]
['C#' 'C++' 'Go' 'Kotlin' 'Python' 'R' 'java']


In [14]:
# 순서가 의미를 갖는 경우 - replace 함수 이용
df = pd.DataFrame({'Score' : ['저조', '보통','보통','저조','우수','매우우수']})
scale_mapper = {'저조' : 1, '보통' : 2, '우수' : 3, '매우우수' : 4}
df['encoder'] = df['Score'].replace(scale_mapper)
print(df)

  Score  encoder
0    저조        1
1    보통        2
2    보통        2
3    저조        1
4    우수        3
5  매우우수        4


In [15]:
from sklearn.preprocessing import OrdinalEncoder

features = np.array([['low', 10], ['normal', 20], ['high',15]])

ordinal_encoder = OrdinalEncoder()

print(ordinal_encoder.fit_transform(features))
print(ordinal_encoder.categories_)

[[1. 0.]
 [2. 2.]
 [0. 1.]]
[array(['high', 'low', 'normal'], dtype='<U11'), array(['10', '15', '20'], dtype='<U11')]


In [20]:
# 분류 모델을 이용한 결측값 대체
from sklearn.neighbors import KNeighborsClassifier

# 훈련 데이터 생성
X = np.array([[0,2.10,1.45], [1,1.18,1.33],[0,1.22,1.27],[1,-0.21, -1.19]])

# 예측에 사용할 데이터
X_with_nan = np.array([[np.nan, 0.87, 0.31], [np.nan, -0.67, -0.22]])

# KNN 학습기 생성
clf = KNeighborsClassifier(3, weights='distance')
#첫번째 데이터를 label 로 하고 나머지 데이터를 feature 로 설정해서 훈련
trained_model = clf.fit(X[:, 1:], X[:, 0])

# 예측
impute_values = trained_model.predict(X_with_nan[:, 1:])
print(impute_values)

# 예측한 데이터와 원본 데이터를 합치기
X_with_imputed = np.hstack((impute_values.reshape(-1,1), X_with_nan[:, 1:]))
print(X_with_imputed)

# 결측치를 대체한 데이터와 훈련에 사용한 데이터 합치기
result = np.vstack((X_with_imputed, X))
result

[0. 1.]
[[ 0.    0.87  0.31]
 [ 1.   -0.67 -0.22]]


array([[ 0.  ,  0.87,  0.31],
       [ 1.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

In [ ]:
# 가장 많이 나오는 데이터로 대체
from sklearn.impute import SimpleImputer

X_complete = np.vstack((X_with_nan, X))
print(X_complete)
imputer = SimpleImputer(strategy='most_frequent')
imputer.fit_transform(X_complete)

In [23]:
import re

# 매칭 여부를 확인
match = re.match('[0-9]', '1234')

# 패턴에 일치하는 데이터가 있으면 Match 객체를 리턴하고 없으면 None 을 리턴
print(match)

match = re.match('[0-9]', 'abc')
print(match)

match = re.match('\s[0-9]', ' 1234')
# 패턴에 일치하는 데이터가 있으면 Match 객체를 리턴하고 없으면 None 리턴
print(match)

<re.Match object; span=(0, 1), match='1'>
None
<re.Match object; span=(0, 2), match=' 1'>


In [32]:
string = '@안녕하세요 반갑습니다^^ 123 의미없는 숫자.... !!!!'

# 숫자 데이터 제거
p = re.compile('[0-9]+')
result = p.sub('', string)
print(result)

# 특수문제 제거
p = re.compile('\W+')
result = p.sub(' ', result)
print(result)

@안녕하세요 반갑습니다^^  의미없는 숫자.... !!!!
 안녕하세요 반갑습니다 의미없는 숫자 


In [34]:
import unicodedata
import sys

text_data = ['안녕하세요 반갑습니다.', 'My job is Programmer', 'C&C++, C#, Python']

# 구두점 딕셔너리 생성
punctuation =dict.fromkeys(i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P'))

result = [string.translate(punctuation) for string in text_data]
print(result)

['안녕하세요 반갑습니다', 'My job is Programmer', 'CC++ C Python']


In [35]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [41]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

string = 'The science of today is the technology of tomorrow'

print(word_tokenize(string))

string = '''품질은 양보다 중요하다. 한 번의 홈런이 두 번의 2루타보다 낫다.
혁신은 현존하는 수천가지 것들에 아니라고 말하는 것이다.'''

print(sent_tokenize(string))

['The', 'science', 'of', 'today', 'is', 'the', 'technology', 'of', 'tomorrow']
['품질은 양보다 중요하다.', '한 번의 홈런이 두 번의 2루타보다 낫다.', '혁신은 현존하는 수천가지 것들에 아니라고 말하는 것이다.']


In [50]:
# 불용어 제거 - 한글은 불용어 사전이 없어서 불용어 사전을 직접 작성
word_korean = ['1월','2월','3월','4월']
stopwords = ['2월', '5월']
result = [i for i in word_korean if i not in stopwords]
print(result)

from nltk.corpus import stopwords
# 영문은 nltk 에서 기본적인 불용어 사전을 제공
word_english = ['chief', 'the', 'and', 'an', 'president','kenedy', 'move']
result = [w for w in word_english if w not in stopwords.words('english')]
print(result)

# sklearn이 nltk 보다 불용어의 개수가 조금 더 많음
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
result = [w for w in word_english if w not in ENGLISH_STOP_WORDS]
print(result)

['1월', '3월', '4월']
['chief', 'president', 'kenedy', 'move']
['chief', 'president', 'kenedy']


In [58]:
# 영문 어간 추출
from nltk.stem import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.tokenize import word_tokenize

string = "All pythoners have pythoned poorly at lest once person personalization"

#단어 토큰화 - 공백을 기준으로 분할해서 list 로 생성
words = word_tokenize(string)
print(words)

ps_stemmer = PorterStemmer()
# 어간 추출
for w in words:
    print(ps_stemmer.stem(w), end="\t")

['All', 'pythoners', 'have', 'pythoned', 'poorly', 'at', 'lest', 'once', 'person', 'personalization']
all	python	have	python	poorli	at	lest	onc	person	person	

In [60]:
# 형태소 분석 - 품사 태깅
# import nltk
# nltk.download('averaged_perceptron_tagger')

# from nltk import pos_tag
# from nltk import word_tokenize

tokens = word_tokenize('The little yellow don barked at the Persian cat')
tags_en = pos_tag(tokens)
# 단어와 품사의 튜플의 리스트로 출력
# 품사(NNP : 고유명사, NN: 명사, RB : 부사, VBD : 동사, VBG: 동사, 동명사, 현재분사, JJ : 형용사)
print(tags_en)
print([word for word, tag in tags_en if tag in ['NN','NNP']])


[('The', 'DT'), ('little', 'JJ'), ('yellow', 'JJ'), ('don', 'NN'), ('barked', 'VBD'), ('at', 'IN'), ('the', 'DT'), ('Persian', 'JJ'), ('cat', 'NN')]
['don', 'cat']


In [1]:
text = '''태양계는 지금으로부터 약 46억년 전 거대한 분자 구름의 일부분이 중력 붕괴를
일으키면서 형성되었다'''
from konlpy.tag import Kkma
kkma = Kkma()
# 문장 분석
print(kkma.sentences(text))
# 단어 분석
print(kkma.nouns(text))
# 형태소 분석
print(kkma.pos(text))

# 다른 형태소 분석기
# 성능은 Kkma가 우수하다고 알려져 있는데 메모리 사용량이 많고 속도가 조금 느림
from konlpy.tag import Hannanum
hannanum = Hannanum()
# 단어 분석
print(hannanum.nouns(text))
# 형태소 분석
print(hannanum.morphs(text))
print(hannanum.pos(text))

['태양계는 지금으로부터 약 46억 년 전 거대한 분자 구름의 일부분이 중력 붕괴를 일으키면서 형성되었다']
['태양계', '지금', '46', '46억년', '억', '년', '전', '거대', '분자', '구름', '일부분', '중력', '붕괴', '형성']
[('태양계', 'NNG'), ('는', 'JX'), ('지금', 'NNG'), ('으로', 'JKM'), ('부터', 'JX'), ('약', 'MDN'), ('46', 'NR'), ('억', 'NR'), ('년', 'NNB'), ('전', 'NNG'), ('거대', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('분자', 'NNG'), ('구름', 'NNG'), ('의', 'JKG'), ('일부분', 'NNG'), ('이', 'JKS'), ('중력', 'NNG'), ('붕괴', 'NNG'), ('를', 'JKO'), ('일으키', 'VV'), ('면서', 'ECE'), ('형성', 'NNG'), ('되', 'XSV'), ('었', 'EPT'), ('다', 'EFN')]


In [8]:
# BoW(Bag of Word - 단어 개수)
from sklearn.feature_extraction.text import CountVectorizer

text_data = np.array([
    'I love Newziland newziland',
    'Sweden is good',
    'I hate Japan',
    'Korea is best place to live'
])

count = CountVectorizer()
bag_of_words = count.fit_transform(text_data)
# print(bag_of_words)
# print(bag_of_words.toarray())

# 특성 이름 확인
print(count.get_feature_names_out())
# 밀집 행렬의 형태로 출력됨
print(bag_of_words.toarray())

['best' 'good' 'hate' 'is' 'japan' 'korea' 'live' 'love' 'newziland'
 'place' 'sweden' 'to']
[[0 0 0 0 0 0 0 1 2 0 0 0]
 [0 1 0 1 0 0 0 0 0 0 1 0]
 [0 0 1 0 1 0 0 0 0 0 0 0]
 [1 0 0 1 0 1 1 0 0 1 0 1]]


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

text_data = np.array([
    'I love korea',
    'I korea love',
    'USA is good',
    'USA beats both',
    'japan is so hot'
])

# tf-idf 객체를 생성
tfidf = TfidfVectorizer()
feature_matrix = tfidf.fit_transform(text_data)
# feature_matrix

print(feature_matrix.toarray())

[[0.         0.         0.         0.         0.         0.
  0.70710678 0.70710678 0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.70710678 0.70710678 0.         0.        ]
 [0.         0.         0.659118   0.         0.53177225 0.
  0.         0.         0.         0.53177225]
 [0.61418897 0.61418897 0.         0.         0.         0.
  0.         0.         0.         0.49552379]
 [0.         0.         0.         0.52335825 0.42224214 0.52335825
  0.         0.         0.52335825 0.        ]]
